# SVM - Climate Sentiment Multiclass Classification
## CS522 Project

### Dataset: 
https://www.kaggle.com/code/luiskalckstein/climate-sentiment-multiclass-classification

### Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import LinearSVC
from Common.DataCenter import data_center
from Common.preprocessor import normalize_preprocessing
import numpy as np

%matplotlib inline

### Text preprocessing

In [2]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train, X_test):
    
    # preprocessing with traditional NLP methodology
    X_train_normalized = normalize_preprocessing(X_train)
    X_test_normalized  = normalize_preprocessing(X_test)
    
    # vectorization
    # Convert texts to vectors by TFIDF
    vectorizer = TfidfVectorizer()
    X_train_vec  = vectorizer.fit_transform(X_train_normalized)
    X_test_vec   = vectorizer.transform(X_test_normalized)
      
    return X_train_vec, X_test_vec

### One-hot encoding, convert the labels to vectors (4 x 1) each

In [3]:
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train, y_test):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    y_test_vec   = mlb.transform(map(str, y_test))
    return y_train_vec, y_test_vec


### Run SVM and evaluate the results

In [4]:
# parameter:  vectorised X and encoded y of training set and test set
def evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec):
    # Run SVM - fit and predict
    SVM             = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
    SVM.fit(X_train_vec, y_train_vec)
    prediction      = SVM.predict(X_test_vec)

    # Evaluate the results
    macro_f1        = f1_score(y_test_vec, prediction, average='macro')
    weighted_f1     = f1_score(y_test_vec, prediction, average='weighted')
    macro_precision = precision_score(y_test_vec, prediction, average='macro')
    macro_recall    = recall_score(y_test_vec, prediction, average='macro')

    return macro_f1, weighted_f1, macro_precision, macro_recall


### Do an experiment

In [5]:
# Parameter: original X,y of training set and test set
def do_experiment(X_train, y_train, X_test, y_test):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)
    
    # Run SVM and evaluate the results
    macro_f1, weighted_f1, macro_precision, macro_recall = \
        evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

    # Show the indicators
    print(" macro_f1: %.4f , weighted_f1: %.4f, macro_precision: %.4f, macro_recall: %.4f" %
          (macro_f1, weighted_f1, macro_precision, macro_recall))


### Main entry

**Load the database and split it into training set, test set, noisy set, validation set**

In [6]:
dc = data_center("twitter_sentiment_data.csv", test_size=8000, noisy_size=8000, validation_size=5000)

print("####################################################")
print("Total data size: ",       dc.get_len())
print("Total train data size: ", dc.get_train_len())
print("Total test data size: ",  dc.get_test_len())

####################################################
Total data size:  41033
Total train data size:  20033
Total test data size:  8000


**Get the test set for evaluation**

In [7]:
X_test, y_test = dc.get_test()


**Run experiments with different training sets, and use the same test set.**

In [8]:
print("-----------------------------------------------")
for size in [1000, 2000, 2500, 4000, 5000, 7500, 10000]:
    # Get a training set without noisy data
    X_train, y_train = dc.get_train(size)
    print("Training set size: %d samples (%.1f%%): " % (len(X_train), len(y_train)/dc.get_train_len()*100))

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

print("-----------------------------------------------")
for size in [(2000, 500), (4000, 1000), (7500, 2500)]:
    # Get a noisy training set
    X_train, y_train = dc.get_train_with_noisy(size[0], size[1])
    print("Noisy training set size: %d samples (%d original, %d noisy)" % (len(y_train), size[0], size[1]))

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

-----------------------------------------------
Training set size: 1000 samples (5.0%): 
 macro_f1: 0.4147 , weighted_f1: 0.5344, macro_precision: 0.6373, macro_recall: 0.3741
Training set size: 2000 samples (10.0%): 
 macro_f1: 0.4659 , weighted_f1: 0.5706, macro_precision: 0.6628, macro_recall: 0.4179
Training set size: 2500 samples (12.5%): 
 macro_f1: 0.4849 , weighted_f1: 0.5816, macro_precision: 0.6667, macro_recall: 0.4357
Training set size: 4000 samples (20.0%): 
 macro_f1: 0.5296 , weighted_f1: 0.6138, macro_precision: 0.6906, macro_recall: 0.4789
Training set size: 5000 samples (25.0%): 
 macro_f1: 0.5392 , weighted_f1: 0.6213, macro_precision: 0.6845, macro_recall: 0.4915
Training set size: 7500 samples (37.4%): 
 macro_f1: 0.5722 , weighted_f1: 0.6437, macro_precision: 0.7057, macro_recall: 0.5192
Training set size: 10000 samples (49.9%): 
 macro_f1: 0.5872 , weighted_f1: 0.6541, macro_precision: 0.7033, macro_recall: 0.5387
-----------------------------------------------
N